# Innodb核心功能

### 内存池：
* 缓冲池（Buffer Pool）：所有的更新或者写操作最先发生在内存中，然后以一定的频率刷新到磁盘，按最近最少使用(LRU)算法更新缓存池中的数据（按页(16KB)读取文件到缓存池中）
* 插入缓存，提高修改性能（Change Buffer）：主要用于更新非聚集索引，当非聚集索引的页不在内存缓存池中时，所有的修改会记录到该Buffer中，然后当对应的索引页加载到内存中后会将该Buffer中的数据更新到对应的索引页
* 两次写缓存，提高可靠性（Doublewrite Buffer）：刷新脏页时，会将脏页数据拷贝到doublewrite缓存中，然后才会将数据写入实际的表文件，因此当最终写入文件时一旦出现问题，则可以使用该缓存中的数据还原数据库
* 重做日志缓存池（Redo Log Buffer）：主要负责记录重做日志信息，每秒钟刷新到磁盘，即使事务没有提交

****

### 主线程循环逻辑：
* 刷新重做日志到磁盘
* 合并插入缓存
* 可能刷新脏页到磁盘
* 切换到后台循环线程
* 删除无用undo页
* 产生检查点

****

### 关键特性：
* 支持ACID原子性事务，实现四种隔离级别
* 支持行锁，非锁定读，通过MVCC控制高并发
* 按聚集索引方式存储组织数据
* 支持外键操作，保障多表级联操作的数据一致性
* 预读取：
    * 随机预读取，当缓存中存在一个区的13个页并且在LRU列表的前端，则会将该区中的所有页都读入缓存，已取消
    * 线性预读取，当一个区中的24个页都被顺序访问过，则会下一个区的所有页

****

### 文件
* 表空间文件：
    * 系统表空间：
        * 该表空间的的存储信息是共享的
        * 包含二次写缓存，修改缓存，undo日志，如果用户没有指定创建索引和表使用独立空间，则这些信息也会存储在系统表空间中
        * ```innodb_data_file_path```：指定innodb表空间文件存储路径，默认所有表共享该空间，文件结尾后缀为ibdataXXX
    * 独立表空间：
        * 包含用户创建的表信息和索引信息
        * ```innodb_file_per_table```：指定每个表使用独立的表空间文件，文件后缀为ibdXXX
    * 通用表空间：
        * 可以将表的数据文件建立在Mysql的data目录之外的目录，文件后缀为ibdXXX
        * ` CREATE TABLE tbl_name ... TABLESPACE [=] tablespace_name or ALTER TABLE tbl_name TABLESPACE [=] tablespace_name`：创建通用表空间
    * 临时表空间：
        * 用于存储临时表信息
        * `innodb_temp_data_file_path=ibtmp1:12M:autoextend:max:500M`：指定临时表空间的位置、大小等信息，默认为12M，文件后缀为ibtmpXXX
        * 临时表空间在Mysql正常关闭以及终止初始化过程中会被删除，每次Mysql重启会重新创建（创建失败则会导致Mysql启动不了），但是Mysql异常关闭时，临时表空间不会被删除
* 表结构文件：定义表的结构，与存储引擎无关，以后缀frm结尾
* undo日志文件：
    * 用来记录老数据的快照，实现MVCC
    * insert undo日志：在事务回滚时使用，所以一旦事务提交，就会删除日志中的数据
    * update undo日志：一致性读会使用该undo类型的日志数据，所以只有当数据库中没有事务在使用在日志中的快照数据时，才会被删除
    * undo日志存在在回滚段中，而回滚段是单独存储在undo表空间和临时表空间中（undo表空间即将被废弃，临时表空间中的undo日志记录的是操作临时表的数据）
* redo日志文件：
    * 记录innodb的事务日志，用于断电后故障恢复，文件名为ib_logfileXXX
    * 采取循环覆盖写
    * 记录innodb本身物理页的改变，事务进行过程中也会记录
    * ```innodb_log_file_size```：指定重做日志文件大小
    * ```innodb_log_files_in_group```：指定日志文件组中重做日志的数量
    * ```innodb_mirrored_log_groups```：指定日志镜像文件组，即日志文件组的数量
    * ```innodb_log_group_home_dir```：指定日志文件组路径
    * ```innodb_flush_log_at_trx_commit```：设置重做日志缓存刷新规则
        * 0表示不根据commit提交，只根据时间刷新
        * 1表示commit时刷新
        * 2表示异步刷新也就是说commit的时候不一定会刷新只会有这么一个动作
 
**** 
 
### 逻辑存储
* 表空间由段、区、页组成
    <img src='../images/mysql/存储引擎存储空间.png' width='300px'>
* 段由数据段，索引段，回滚段组成，数据段为B+树的叶子节点，索引段为B+树的非叶子节点，大的数据段每次最多可以申请4个区
* 区由64个连续的页组成，每个页大小16KB，每个区1MB，页是最小的存储单位
* 页包含数据页，索引页，undo页，系统页，事务数据页，插入缓存位图页，插入缓存空闲列表页，未压缩的二进制大对象页，压缩的二进制大对象页
    <img src='../images/mysql/存储引擎页结构.png' width='150px'>
* 行数据存储在页中的User Records中，页中包含多行数据，最多包含7992行数据，行格式有如下四种：
    * Redundant和Compact：
        * 数据除了用户定义的列以外，还包括DB_TRX_ID和DB_ROLL_PTR，如果没有定义主键则还包括DB_ROW_ID
        * 不会截断varchar类型的尾部空格
        * 变长、固定长度的字段长度大于等于768字节时，前768字节直接存储，剩下的存储到未压缩的二进制大对象页中
        * 索引前缀最大大小为768字节
        * 每行数据的可变类型大小不能超过65535字节
        * Redundant不同点：
            * 数据头用于连接连续的记录，大小为6字节，支持行锁
            * 对于varchar类型的NULL值不占用空间，char类型的NULL需要占用存储空间
        * Compact不同点：
            * 数据头用于连接连续的记录，大小为5字节，支持行锁，并且会标识NULL列的索引
            * varchar和char类型为NULL值的列不占用存储空间
            * 相比Redundant行格式，能节省20%的空间，但是更占用CPU
    * Dynamic和Compressed：
        * 变长字段存储根据数据的大小动态的选择直接存储数据还是存储在BLOB页中，如果少于40字节则直接存储；否则只存储前20字节指针，其他存储在BLOB页中
        * 固定长度的字段长度大于等于768字节时，会按变长字段处理
        * 索引前缀最大大小为3072字节
        * Dynamic不同点：
            * 支持独立表空间、通用表空间和系统表空间
            * 默认的行格式
        * Compressed不同点：
            * 支持独立表空间和通用表空间
            * 除了动态存储数据外，还会对大数据对象进行压缩，因此相比Dynamic行格式，更省空间，但是更占用CPU

****

### 数据完整性
* 实体完整性：保证表中有一个主键
* 域（列）完整性：保证数据的值满足特定的条件
* 参照完整性：保证两张表之间的关系
* 通过约束来实现数据的完整性，支持的约束：
    * Primary Key约束
    * Foreign Key约束：可以指定DELETE和UPDATE的级联操作，及时触发
        * CASCADE：当父表发生DELETE或UPDATE操作时，子表也会被DELETE或UPDATE
        * SET NULL：当父表发生DELETE或UPDATE操作时，子表会将对应列更新为NULL
        * NO ACTION：当父表发生DELETE或UPDATE操作时，抛出错误，不允许这类操作
        * RESTRICT：当父表发生DELETE或UPDATE操作时，抛出错误，不允许这类操作，外键约束级联操作的默认值
    * Unique Key约束、NOT NULL、Default、ENUM约束、SET约束
    * 触发器
        * 在数据更新，插入，删除之前或之后做约束检查，提供高级的完整性约束检查

****

### 数据索引，提速查询
* B+树索引只能找到对应的数据页，然后将数据页加载到内存中查找具体的行数据
* 聚集索引：
    * 一张表只能建立一个聚集索引，一般是将主键构建成一颗B+树
    * 索引页存放键值以及指向数据页的偏移量
    * 聚集索引对于主键的排序查找和范围查找非常快：叶子节点通过双向链表链接，可以快速定位主键；索引页记录了叶子节点的范围，可以快速找出范围内的数据
    * 聚集索引的创建：创建临时表，把数据导入临时表，删除原表，将临时表重命名为原表
    * 查询数据时，通过叶节点指针顺序读取数据
* 非聚集索引（辅助索引）：
    * 一张表可以有多个非聚集索引
    * 叶节点包含对应的键值以及指向聚集索引主键的指针
    * 非聚集索引查找过程：先通过键值找到非聚集索引的叶子节点，然后通过指向聚集索引主键的指针在聚集索引中查找对应的叶节点，最后取出行数据
    * 非聚集索引的创建：创建过程中，对表加S锁，只能读不能写，不需要重建表
    * 非聚集索引的删除：直接将非聚集索引占用的空间标记为可用
    * 查询数据时，通过叶节点的主键指针随机读取数据
* B+树索引使用情况分析（何时建立非聚集索引）
    * 低选择性意味着列值可选范围很小，数据多，如性别，不推荐使用
    * 高选择性意味着列值可选范围很大，数据少，几乎没重复性，例如姓名，推荐使用
    * 但是在高选择性列上查询大范围数据时（全部数据的20%），查询优化器会直接使用全表扫描
* 自适应HASH索引，提高查询性能：根据索引的查询频率自动建立HASH索引，只能用于等值比较，且采取链表方式解决HASH冲突
* 全文索引：
    * 每个记录有对应的记录ID，全文索引是词到记录ID的映射
    * 全文索引的更新是先插入到缓存（查询结果会合并该缓存中的结果集），然后一次性更新到索引表中，避免并发性问题以及频繁的插入操作
    * 全文索引记录被删除后，记录ID并不会在索引表中删除，只是在另外一个表中记录该ID（用该表中的ID过滤结果集）
    * 只有提交完的记录才可以在索引中被查找出来
    * SQL：`SELECT COUNT(*) FROM opening_lines WHERE MATCH(opening_line) AGAINST('Ishmael');`
    * Innodb全文索引详情：https://dev.mysql.com/doc/refman/8.0/en/innodb-fulltext-index.html
* Innodb更新索引的策略：https://dev.mysql.com/doc/refman/8.0/en/sorted-index-builds.html

****

### MVCC多版本并发控制，一致性非锁定读
* DB_TRX_ID：记录上次插入或更新该行数据的事务ID，删除操作是一种特殊的更新操作
* DB_ROLL_PTR：用来指向undo日志中的老版本行数据
* DB_ROW_ID：隐藏行ID，如果表没有主键，则该字段会被做为主键
* 事务中的更新操作开始之前，会将老数据先写入undo日志，然后再操作数据
* *注意，这里所说的时间并不是真实的系统时间，而是事务序列号（系统版本号），每开启一个事务，该号会自动自增*
* SELECT操作：要求行数据的创建时间必须早于当前事务的时间并且行数据的删除时间要么未定义要么要晚于当前事务的时间
* INSERT操作：将当前事务的时间作为新插入的行数据版本号
* DELETE操作：将当前事务的时间添加为行数据的过期时间
    * 物理删除发生的前提条件是在undo日志中完成了对应的记录
* UPDATE操作：将当前事务的时间添加为新行数据的创建时间，同时将当前事务的时间添加为旧的行数据的过期时间

****

### 数据锁，保证并发一致性，实现事务隔离性
* 标准的行级锁类型：
    * 共享锁（S Lock），允许事务读一行数据，同时允许多个事务获取共享锁读取数据
    * 排他锁（X Lock），允许事物删除或更新一行数据
* 意向锁类型：表级锁，表明事务稍后对表中的行所需的锁类型
    * 意向共享锁（IS Lock），事务想获得一个表中某几行的共享锁，与X Lock互斥
    * 意向排他锁（IX Lock），事务想获得一个表中某几行的排他锁，与S Lock和X Lock互斥
    * 事务获取行数据的共享锁之前，必须先获得意向共享锁；事务获取行数据的排他锁之前，必须先获得意向排他锁
    * IS Lock和IX Lock是相互兼容的，且意向锁不会阻塞除全表扫描以外的任何操作
* 加锁读：
    * SELECT ... FOR UPDATE：对读取的数据加IX锁，会阻塞其他事务在该数据上加S Lock和X Lock，但是不会阻塞一致性非锁定读的操作
    * SELECT ... LOCK IN SHARE MODE | SELECT ... FOR SHARE：对读取的数据加IS锁，会阻塞其他事务在该数据上加X Lock
        * 对于外键的插入或更新，使用是SELECT ... LOCK IN SHARE MODE模式锁定父表记录，然后在操作子表
    * 使用上述两种操作时必须关闭事务自动提交功能，例如加上BEGIN或START TRANSACTION或SET AUTOCOMMIT=0操作
    * NOWAIT和SKIP LOCKED两种避免等待锁的选项
        * NOWAIT执行时，如果查询的记录有锁，则会报错
        * SKIP LOCKED执行时，会在结果集中排除那些有锁的记录
* 行锁：行锁是加在索引上的，会将索引行锁住，如果没有显示指定索引，则会使用MYSQL隐式创建的索引
* 间隙锁：
    * 间隙锁定是锁定索引记录之间的间隙，或锁定在第一个或最后一个索引记录之前的间隙上，该范围间隙内的数据都会被锁定
    * 如果谓词列不是索引列或者唯一索引列，则查询会添加间隙锁
    * 冲突的锁可以通过不同的事务保持在一个间隙上，也就是间隙锁之间是兼容的（读写共存）
    * 间隙锁唯一的目的是阻止其他事务往间隙中插入新的数据
* Next-Key Locks：行锁和间隙锁的组合，会使用行锁锁定索引记录，同时用间隙锁锁定该索引记录以前的范围，确保不出现幻读
* 插入意向锁：间隙锁的一种，保障插入同一间隙时，如果各事务插入的位置不同，无需阻塞对方，插入意向锁获得时间早于排他锁
* AUTO-INC Locks：如果使用自增长主键，插入数据时会使用该锁
    * 该锁机制不是在事务提交时释放，而是SQL语句操作完成后立即释放
    * 大批量插入数据时，并发性能会下降，因为后面的插入操作必须等待前面的插入操作完成
    * 新版本有AUTO-INC Locking锁机制和通过互斥量累加计数器两种操作模式，可通过参数innodb_aotu_lock_mode进行配置
* 谓词锁：解决空间索引数据的问题
* 查看当前状态下锁的状态：
    * information_schema.innodb_trx：当前事务的状态信息，分析锁资源使用情况
    * information_schema.innodb_locks：直接查看锁资源的状态信息
    * information_schema.innodb_lock_waits：查看等待锁资源的事务ID信息
* 加锁分析：https://dev.mysql.com/doc/refman/8.0/en/innodb-locks-set.html
* innodb的限制：https://dev.mysql.com/doc/refman/8.0/en/innodb-restrictions.html

****

### 事务，保证数据库ACID
* 事务的隔离级别
    * READ UNCOMMITTED
    * READ COMMITTED
        * 默认使用MVCC，读取被锁定行的最新快照版本
        * 加锁读或者其他操作只会锁定索引行记录（如果有索引则只根据索引锁定行记录），间隙锁和Next Key Lock是关闭的
    * REPEATABLE READ：
        * 默认隔离级别
        * 默认使用MVCC，读取事务开始时被锁定行的快照版本
        * 加锁读或者其他操作，如果有索引且是等值查找则使用行锁，否则使用间隙锁或者Next Key Lock锁定范围
    * SERIALIZABLE：
        * 将SELECT语句转换成SELECT ... FOR SHARE模式，保证串行化
        * 主要用于InnoDB存储引擎的分布式事务
* 原子性、一致性、持久性通过redo和undo实现
    * REDO：事务开始时，将该事物的日志序列号（LSN）记录入重做日志的缓存；事务执行时，将事务的操作记录到重做日志缓存；事务提交时，将重做日志缓存写入磁盘，即写数据前先写日志
    * UNDO：对于数据库的修改，不仅记录重做日志，而且还会记录undo日志，undo日志存储在共享表空间中的undo段，undo日志记录事务的反向操作
* 事务控制语句：
    * START TRANSACTION|BEGIN：存储过程中只能使用START TRANSACTION显示开启一段事务操作
    * COMMIT|COMMIT WORK & ROLLBACK|ROLLBACK WORK：
        * COMMIT WORK控制事务提交后的行为，默认情况下与COMMIT行为一致，可通过completion_type设置
        * ROLLBACK WORK与COMMIT WORK概念一致
    * SAVEPOINT identifier & RELEASE SAVEPOINT identifier & ROLLBACK TO [SAVEPOINT] identifier：
        * ROLLBACK TO [SAVEPOINT] identifier并不代表事务结束，还是需要显示的调用COMMIT|ROLLBACK操作结束事务
    * SET TRANSACTION
* XA事务支持分布式事务，XA事务运行不同数据库之间的分布式事务，只要每个参与节点都支持XA事务
    * XA事务是存储引擎提供的支持，因此不同存储引擎之间的事务也会通过XA事务协调
    * 处理分布式事务时，隔离级别必须设置为SERIALIZABLE


****

# Innodb系统参数
* 配置详细：
    * Innodb本身的配置：https://dev.mysql.com/doc/refman/8.0/en/innodb-configuration.html
    * 表空间的配置：https://dev.mysql.com/doc/refman/8.0/en/innodb-tablespace.html
* innodb_read_io_threads，innodb_write_io_threads：指定读写线程数量
* innodb_buffer_pool_size：指定缓冲池大小
* innodb_log_buffer_size：指定重做日志缓冲池大小
* innodb_additianal_mem_pool_size：指定额外的内存池大小
* innodb_max_dirty_pages_pct：最大脏页比，脏页超过这个比值就会触发脏页刷盘
* innodb_io_capacity：表示磁盘IO吞吐量
* innodb_adaptive_flushing：自适应刷新，自动判断刷新脏页的数量
* skip_innodb_doublewrite：设置两次写功能
* innodb_adaptive_hash_index：设置自适应HASH索引
* innodb_adaptive_hash_index_parts：设置自适应HASH分区数量，最大值为512
* innodb_force_recovery：指定恢复时到操作，有1-6个选项值
* innodb_file_format：指定innodb文件格式
* innodb_read_ahead_threshold：指定一个区中的多少页被顺序访问时才触发线性预读取
* innodb_autoinc_lock_mode：设置插入自增长主键数据时的锁类型
    * Simple inserts简单插入操作：指插入之间就能知道确切的插入行数
    * Bulk inserts批量插入操作：指插入之间不知道确切的插入行数
    * Mixed-mode inserts混合插入模式：指有一部分主键是自增长的，有一部分主键是确定的
    * 0表示使用AUTO-INC Locking锁机制
    * 1表示针对Simple inserts使用互斥量进行累加主键值，对Bulk inserts使用AUTO-INC Locking锁机制
    * 2表示针对所有插入操作都使用互斥量进行累加主键值，并发时可能出现自增长的值不是连续的，因此恢复时要基于行数据恢复，不要基于语句恢复
* innodb_lock_wait_timeout：设置锁的阻塞等待时间，默认50s，避免死锁
* innodb_rollback_on_timeout：设置阻塞等待时间超时时是否回滚事务，默认关闭
* innodb_flush_log_at_trx_commit：设置重做日志缓存刷新策略
* innodb_support_xa：查询innodb是否开启XA事务
* 缓存池命中率计算：<img src='../images/mysql/缓存池命中率计算公式.png' width='300px'>